<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

**1D integral with vector output**:

$$ \int^1_0 \mathbf f(x) \text dx $$
Where ``f(x) = x^n`` and ``n = np.arange(10)`` is a vector. Since no rule is
specified, the default "gk21" is used, which corresponds to Gauss-Kronrod
integration with 21 nodes.


In [ ]:
import numpy as np
from scipy.integrate import cubature
def f(x, n):
   # Make sure x and n are broadcastable
   return x[:, np.newaxis]**n[np.newaxis, :]
res = cubature(
    f,
    a=[0],
    b=[1],
    args=(np.arange(10),),
)
res.estimate

 array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ])

**7D integral with arbitrary-shaped array output**
```

f(x) = cos(2*pi*r + alphas @ x)

```
for some ``r`` and ``alphas``, and the integral is performed over the unit
hybercube, $[0, 1]^7$. Since the integral is in a moderate number of
dimensions, "genz-malik" is used rather than the default "gauss-kronrod" to
avoid constructing a product rule with $21^7 \approx 2 \times 10^9$ nodes.


In [ ]:
import numpy as np
from scipy.integrate import cubature
def f(x, r, alphas):
    # f(x) = cos(2*pi*r + alphas @ x)
    # Need to allow r and alphas to be arbitrary shape
    npoints, ndim = x.shape[0], x.shape[-1]
    alphas = alphas[np.newaxis, ...]
    x = x.reshape(npoints, *([1]*(len(alphas.shape) - 1)), ndim)
    return np.cos(2*np.pi*r + np.sum(alphas * x, axis=-1))
rng = np.random.default_rng()
r, alphas = rng.random((2, 3)), rng.random((2, 3, 7))
res = cubature(
    f=f,
    a=np.array([0, 0, 0, 0, 0, 0, 0]),
    b=np.array([1, 1, 1, 1, 1, 1, 1]),
    rtol=1e-5,
    rule="genz-malik",
    args=(r, alphas),
)
res.estimate

 array([[-0.79812452,  0.35246913, -0.52273628],
        [ 0.88392779,  0.59139899,  0.41895111]])

**Parallel computation with** `workers`:


In [ ]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    res = cubature(
        f=f,
        a=np.array([0, 0, 0, 0, 0, 0, 0]),
        b=np.array([1, 1, 1, 1, 1, 1, 1]),
        rtol=1e-5,
        rule="genz-malik",
        args=(r, alphas),
        workers=executor.map,
     )
res.estimate

 array([[-0.79812452,  0.35246913, -0.52273628],
        [ 0.88392779,  0.59139899,  0.41895111]])

**2D integral with infinite limits**:

$$ \int^{ \infty }_{ -\infty } \int^{ \infty }_{ -\infty } e^{-x^2-y^2} \text dy \text dx $$

In [ ]:
def gaussian(x):
    return np.exp(-np.sum(x**2, axis=-1))
res = cubature(gaussian, [-np.inf, -np.inf], [np.inf, np.inf])
res.estimate

 3.1415926

**1D integral with singularities avoided using** `points`:

$$ \int^{ 1 }_{ -1 } \frac{\sin(x)}{x} \text dx $$
It is necessary to use the `points` parameter to avoid evaluating `f` at the origin.


In [ ]:
def sinc(x):
    return np.sin(x)/x
res = cubature(sinc, [-1], [1], points=[[0]])
res.estimate

 1.8921661